In [69]:
#import thinkplot
#import thinkstats2
import pandas as pd
import numpy as np
import math
import scipy.stats as ss
import thinkplot
import thinkstats2
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (15,5)

# Assignment 3 - Basic Predictions and Regression

## Questions

### Part 1 - Election Prediction

Suppose you are looking at an election in a fictional province. There are 7 total elctoral districts, and the winner in each district is determined by a first-past-the-post system (what we have in Canada - the most votes wins, regardless of share). There are two parties - the Purples and the Yellows. Whoever controls the most seats will be the ruling party - so in our 2 party scenario, the party who wins 4 or more of the districts will govern. There is an election every year, they love voting. 

Recent polling indicating the expected vote share in each district is shown in the "dist_polls" table below. These values are a composite of several polls that the experts have combined and weighted. The "Purple" values show the expected vote share of the Purple party, along with the variance of that expectation and the number of polls that were combined to get that result.  

As well, research has shown that the vote distribution is impacted by voter turnout. In general, the more people vote, the more the vote split shifts towards the Yellow party. We have data on past elections and the results, we expect that the turnout will be in line with the past elections - or more specifically, we have no reason to expect it to differ. This impact is measured in the table in code below - that table shows the voter turnout, in a percentage, as well as the change in the Yellow party's vote share (also in percentage) as compared to the polling averages. For example, if one row showed "52" and ".8", that would mean that voter turnout was 52%, and the Yellow party got .8% higher of a vote share than the polling showed. 

<b>What is the probability that the Purple Party controls the government after the election?</b>

<b>Note:</b> the errors and confidence intervals are not totally trivial. As part of the written answer, offer an evaluation of your confidence in the prediction, and why you think that. This is not a question with one specific error, your estimation will have some expected errors, somewhere. You may not have the tools to calculate it all the way through, that's fine. 

### Question 1 - Your Answer in English

Please fill in (and extend if required) the list here to explain what you did. There are multiple reasonable things you could do to approach this, so please note what you did here. For most people I assume this will be about 3-5 statements - you don't need to explain the internals of things we covered (e.g. if there's a hypothesis test, you don't need to explain how that works), just how you structured your approach to the problem. 

<ul>
<li>
<li>
<li>
<li> <b> What do you think about the error/accuracy:</b>
</ul>

##### Setup Poll Data

The dataframe "dist_polls" contains all of the polls for each seat. Each value is expressed as expected vote share (as a ratio) for the <b>Purple</b> party. The Yellow party can be safely assumed to get the rest of the votes. 

In [70]:
# Please don't edit this part. 
# Setup polling data. 
districts = [1,2,3,4,5,6,7]
dist_polls = pd.DataFrame(districts, columns={"district"})

dist_polls["Poll_1"] = [.55, .49, .51, .6, .41, .46, .54]
dist_polls["Poll_2"] = [.53, .51, .51, .62, .44, .48, .53]
dist_polls["Poll_3"] = [.51, .49, .53, .61, .42, .46, .52]
dist_polls["Poll_4"] = [.47, .48, .51, .54, .45, .45, .51]
dist_polls["Poll_5"] = [.61, .52, .49, .73, .44, .51, .53]
dist_polls["Poll_6"] = [.54, .45, .51, .61, .47, .52, .52]
dist_polls["Poll_7"] = [.55, .47, .5, .56, .47, .46, .56]
dist_polls["Poll_8"] = [.53, .49, .51, .55, .43, .49, .55]
dist_polls["Poll_9"] = [.57, .39, .52, .57, .53, .43, .53]


dist_polls.head()

,district,Poll_1,Poll_2,Poll_3,Poll_4,Poll_5,Poll_6,Poll_7,Poll_8,Poll_9
0,1,0.55,0.53,0.51,0.47,0.61,0.54,0.55,0.53,0.57
1,2,0.49,0.51,0.49,0.48,0.52,0.45,0.47,0.49,0.39
2,3,0.51,0.51,0.53,0.51,0.49,0.51,0.50,0.51,0.52
3,4,0.60,0.62,0.61,0.54,0.73,0.61,0.56,0.55,0.57
4,5,0.41,0.44,0.42,0.45,0.44,0.47,0.47,0.43,0.53


##### Setup Turnout Data

The dataframe "past_vte_table" shows the voter turnout, along with the impact on the votes counted for the <b>Yellow party</b>, all expressed as percentages. For example, if in one row the turnout is .45 and the Yellow_improvement is -.04, that means that 45% of the populace turned out to vote, and the Yellow party got 4% fewer votes than polling indicated. 

In [71]:
# Please don't edit this part. 
# Setup vote data. 
voter_turnout_history = [.53, .51, .48, .55, .54, .59, .49, .57, .56]
past_vote_table = pd.DataFrame(voter_turnout_history, columns={"voter_turn_percentage"})
past_vote_table["Yellow_improvement"] = [.012, .023, -.017, .031, .030, -.004, -.03, .042, .029]
past_vote_table["year"] = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
past_vote_table.head()

,voter_turn_percentage,Yellow_improvement,year
0,0.53,0.012,2013
1,0.51,0.023,2014
2,0.48,-0.017,2015
3,0.55,0.031,2016
4,0.54,0.030,2017


## Start Your Work

### Part 1 - Election

In [72]:
# Do some stuff

### Part 2 - Regression

<b>Use the data provided to try to predict the wage. </b>

The data is from FIFA rankings for players. You don't need to know anything about soccer or video games for this, so if these values are meaningless to you, just treat them as numbers and you'll be fine. All of the features are rankings are evaluations of how good different soccar players are at different skills.

#### Answer in English

Please fill in (and extend if required) the list here to explain what you did. There are multiple reasonable things you could do to approach this, so please note what you did here. For most people I assume this will be about 3-5 statements - you don't need to explain the internals of things we covered (e.g. if there's a hypothesis test, you don't need to explain how that works), just how you structured your approach to the problem. 

<ul>
<li>
<li>
<li>
</ul>

In [73]:
df = pd.read_csv("players_20_2.csv")
df.head()

,wage_eur,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,...,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle
0,565000,88,95,70,92,88,97,93,94,92,...,94,48,40,94,94,75,96,33,37,26
1,405000,84,94,89,83,87,89,81,76,77,...,93,63,29,95,82,85,95,28,32,24
2,290000,87,87,62,87,87,96,88,87,81,...,84,51,36,87,90,90,94,27,26,29
3,125000,13,11,15,43,13,12,13,14,40,...,12,34,19,11,65,11,68,27,12,18
4,470000,81,84,61,89,83,95,83,79,83,...,80,54,41,87,89,88,91,34,27,22


In [74]:
#Y data is simple, do that first
y = np.array(df["wage_eur"]).reshape(-1,1)
y.shape

(18278, 1)

In [75]:
#Get a new df with only the features we'll use
df_ = df.drop(columns={"wage_eur"})
df_.head()

,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,...,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle
0,88,95,70,92,88,97,93,94,92,96,...,94,48,40,94,94,75,96,33,37,26
1,84,94,89,83,87,89,81,76,77,92,...,93,63,29,95,82,85,95,28,32,24
2,87,87,62,87,87,96,88,87,81,95,...,84,51,36,87,90,90,94,27,26,29
3,13,11,15,43,13,12,13,14,40,30,...,12,34,19,11,65,11,68,27,12,18
4,81,84,61,89,83,95,83,79,83,94,...,80,54,41,87,89,88,91,34,27,22


In [76]:
#Make that df into an array. 
x = np.array(df_)
x.shape

(18278, 29)

In [77]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Function will check the VIF of each variable in a DF and return the results in another DF
def calc_vif(X):
    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [78]:
#Check VIFs
df_vif = calc_vif(df_)
df_vif.sort_values(by= 'VIF', ascending=False)

,variables,VIF
9,skill_ball_control,258.956301
3,attacking_short_passing,228.179747
10,movement_acceleration,177.056230
27,defending_standing_tackle,165.808035
5,skill_dribbling,164.947533
11,movement_sprint_speed,149.297858
28,defending_sliding_tackle,135.107417
13,movement_reactions,102.412159
12,movement_agility,96.879998
8,skill_long_passing,96.163815


In [79]:
df2 = df_.drop(['skill_ball_control','attacking_short_passing','movement_acceleration',
               'skill_dribbling','defending_sliding_tackle','movement_reactions'],axis=1)

In [80]:
df_vif2 = calc_vif(df2)
df_vif2.sort_values(by= 'VIF', ascending=False)

,variables,VIF
8,movement_agility,89.360246
22,defending_standing_tackle,77.083255
1,attacking_finishing,71.979690
20,mentality_composure,71.465074
17,mentality_positioning,70.905573
10,power_shot_power,70.540146
16,mentality_interceptions,68.879050
14,power_long_shots,64.385388
7,movement_sprint_speed,64.156398
18,mentality_vision,62.132342


In [81]:
df3 = df2.drop(['movement_agility','defending_standing_tackle','attacking_finishing',
               'mentality_composure','mentality_positioning','power_shot_power', 'mentality_interceptions',
               'power_long_shots','movement_sprint_speed','mentality_vision','skill_long_passing',
               'power_stamina','movement_balance','skill_curve','mentality_penalties'],axis=1)






In [82]:
df_vif3 = calc_vif(df3)
df_vif3.sort_values(by= 'VIF', ascending=False)

,variables,VIF
6,mentality_aggression,37.057444
1,attacking_heading_accuracy,29.850998
5,power_strength,29.305086
0,attacking_crossing,26.933805
4,power_jumping,26.022986
2,attacking_volleys,25.836807
3,skill_fk_accuracy,22.515484
7,defending_marking,19.282565


In [88]:
#Statsmodels.
import statsmodels.formula.api as smf

In [89]:
formula = ('wage_eur ~ defending_marking + skill_fk_accuracy + attacking_volleys +power_jumping + power_jumping + power_strength + mentality_aggression +'
               'attacking_heading_accuracy + mentality_composure + mentality_positioning + power_shot_power + mentality_interceptions +' 
               'power_long_shots + movement_sprint_speed + mentality_vision + skill_long_passing +'
               'power_stamina + movement_balance + skill_curve + mentality_penalties +'
               'skill_ball_control + attacking_short_passing + movement_acceleration +'
               'movement_agility + defending_standing_tackle + attacking_finishing + mentality_composure +'
               'skill_dribbling + defending_sliding_tackle + movement_reactions')
results = smf.ols(formula, data=df).fit()
results.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               wage_eur   R-squared:                       0.282
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     256.3
Date:                Thu, 24 Nov 2022   Prob (F-statistic):               0.00
Time:                        10:57:15   Log-Likelihood:            -2.0512e+05
No. Observations:               18278   AIC:                         4.103e+05
Df Residuals:                   18249   BIC:                         4.105e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -6.714e+04   1609.568    -41.713      0.000   -7.03e+04    -6.4e+04
defending_marking             -8.9458     18.622     -0.480      0.631     -45.446      27.555
skill_fk_accuracy            -53.1989     17.091     -3.113      0.002     -86.698     -19.700
attacking_volleys            100.1961     20.196      4.961      0.000      60.611     139.781
power_jumping                 56.3913     13.920      4.051      0.000      29.107      83.675
power_strength               -35.6355     17.101     -2.084      0.037     -69.156      -2.115
mentality_aggression         -66.1295     15.206     -4.349      0.000     -95.935     -36.324
attacking_heading_accuracy    -0.5929     17.368     -0.034      0.973     -34.635      33.449
mentality_composure          219.0082     21.157     10.352      0.000     177.539     260.478
mentality_positioning        -92.8767     21.658     -4.288      0.000    -135.329     -50.425
power_shot_power             145.5565     20.159      7.220      0.000     106.043     185.070
mentality_interceptions      -35.8647     22.604     -1.587      0.113     -80.172       8.442
power_long_shots            -190.9312     21.892     -8.722      0.000    -233.841    -148.021
movement_sprint_speed         64.6265     24.721      2.614      0.009      16.171     113.082
mentality_vision             133.9223     19.852      6.746      0.000      95.010     172.834
skill_long_passing            -0.5190     23.841     -0.022      0.983     -47.249      46.211
power_stamina                -96.8127     16.128     -6.003      0.000    -128.425     -65.200
movement_balance             -38.4763     17.947     -2.144      0.032     -73.653      -3.299
skill_curve                   45.4525     18.896      2.405      0.016       8.414      82.491
mentality_penalties          -22.2156     19.011     -1.169      0.243     -59.479      15.047
skill_ball_control            78.2621     35.460      2.207      0.027       8.757     147.767
attacking_short_passing       93.0280     33.519      2.775      0.006      27.327     158.729
movement_acceleration         31.3559     27.052      1.159      0.246     -21.668      84.380
movement_agility            -100.0578     20.262     -4.938      0.000    -139.774     -60.342
defending_standing_tackle    107.1299     32.989      3.247      0.001      42.469     171.791
attacking_finishing           27.4331     23.196      1.183      0.237     -18.033      72.899
skill_dribbling               10.0940     28.724      0.351      0.725     -46.208      66.396
defending_sliding_tackle     -40.9613     30.945     -1.324      0.186    -101.617      19.695
movement_reactions           918.9718     22.631     40.606      0.000     874.612     963.331
===========================================

In [92]:



formula2 = ('wage_eur ~ power_shot_power +  attacking_heading_accuracy + power_strength + '
               'attacking_crossing + power_jumping + attacking_volleys + skill_fk_accuracy + defending_marking + '
               'skill_curve + movement_balance + power_stamina')
results2 = smf.ols(formula2, data=df).fit()
results2.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               wage_eur   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     264.3
Date:                Thu, 24 Nov 2022   Prob (F-statistic):               0.00
Time:                        11:35:23   Log-Likelihood:            -2.0680e+05
No. Observations:               18278   AIC:                         4.136e+05
Df Residuals:                   18266   BIC:                         4.137e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -3.167e+04   1497.718    -21.144      0.000   -3.46e+04   -2.87e+04
power_shot_power             413.3138     17.947     23.030      0.000     378.136     448.491
attacking_heading_accuracy  -101.9264     14.915     -6.834      0.000    -131.161     -72.692
power_strength               112.9253     17.822      6.336      0.000      77.993     147.858
attacking_crossing            97.4326     17.383      5.605      0.000      63.361     131.505
power_jumping                157.7290     14.468     10.902      0.000     129.370     186.088
attacking_volleys            101.7630     18.851      5.398      0.000      64.814     138.712
skill_fk_accuracy            -84.8736     17.449     -4.864      0.000    -119.075     -50.672
defending_marking            158.2658     11.659     13.574      0.000     135.412     181.119
skill_curve                   92.8041     20.972      4.425      0.000      51.697     133.911
movement_balance             -81.6093     17.155     -4.757      0.000    -115.235     -47.984
power_stamina               -117.3177     15.938     -7.361      0.000    -148.558     -86.078
==============================================================================
Omnibus:                    25421.436   Durbin-Watson:                   0.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9900439.305
Skew:                           8.045   Prob(JB):                         0.00
Kurtosis:                     115.876   Cond. No.                     1.89e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.89e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
formula2 = ('wage_eur ~ power_shot_power +  attacking_heading_accuracy + power_strength + '
               'attacking_crossing + power_jumping + attacking_volleys + skill_fk_accuracy + defending_marking  ')
results2 = smf.ols(formula2, data=df).fit()
results2.summary()

In [91]:
formula = ('wage_eur ~ defending_marking + skill_fk_accuracy + attacking_volleys +power_jumping + power_jumping + power_strength + mentality_aggression +'
               'attacking_heading_accuracy + mentality_composure + mentality_positioning + power_shot_power + mentality_interceptions +' 
               'power_long_shots + movement_sprint_speed + mentality_vision + skill_long_passing +'
               'power_stamina + movement_balance + skill_curve + mentality_penalties +'
               'skill_ball_control + attacking_short_passing + movement_acceleration +'
               'skill_dribbling + defending_sliding_tackle + movement_reactions')
results = smf.ols(formula, data=df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               wage_eur   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     285.0
Date:                Thu, 24 Nov 2022   Prob (F-statistic):               0.00
Time:                        10:57:29   Log-Likelihood:            -2.0513e+05
No. Observations:               18278   AIC:                         4.103e+05
Df Residuals:                   18252   BIC:                         4.105e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -6.756e+04   1605.290    -42.089      0.000   -7.07e+04   -6.44e+04
defending_marking             -0.8273     18.385     -0.045      0.964     -36.864      35.209
skill_fk_accuracy            -53.3504     17.083     -3.123      0.002     -86.835     -19.866
attacking_volleys            105.9269     19.655      5.389      0.000      67.402     144.452
power_jumping                 43.8855     13.771      3.187      0.001      16.892      70.879
power_strength               -26.1210     17.042     -1.533      0.125     -59.525       7.283
mentality_aggression         -64.8990     15.172     -4.277      0.000     -94.639     -35.160
attacking_heading_accuracy    12.9241     17.114      0.755      0.450     -20.621      46.469
mentality_composure          215.8539     21.160     10.201      0.000     174.378     257.330
mentality_positioning        -85.7177     20.685     -4.144      0.000    -126.262     -45.173
power_shot_power             150.2856     20.108      7.474      0.000     110.872     189.699
mentality_interceptions      -15.5356     21.435     -0.725      0.469     -57.550      26.479
power_long_shots            -188.3765     20.974     -8.982      0.000    -229.487    -147.266
movement_sprint_speed         55.5698     24.672      2.252      0.024       7.211     103.928
mentality_vision             128.0928     19.803      6.468      0.000      89.277     166.908
skill_long_passing            -6.7865     23.838     -0.285      0.776     -53.511      39.938
power_stamina               -100.7353     16.073     -6.267      0.000    -132.241     -69.230
movement_balance             -63.5156     17.248     -3.683      0.000     -97.322     -29.709
skill_curve                   39.0637     18.851      2.072      0.038       2.113      76.014
mentality_penalties          -14.9340     18.665     -0.800      0.424     -51.520      21.652
skill_ball_control            68.2134     35.336      1.930      0.054      -1.048     137.474
attacking_short_passing      105.6192     33.462      3.156      0.002      40.030     171.208
movement_acceleration         -0.8926     26.363     -0.034      0.973     -52.567      50.782
skill_dribbling                5.6769     28.654      0.198      0.843     -50.487      61.841
defending_sliding_tackle      36.1593     21.303      1.697      0.090      -5.596      77.914
movement_reactions           908.9689     22.555     40.300      0.000     864.759     953.178
==============================================================================
Omnibus:                    25496.529   Durbin-Watson:                   0.717
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11345658.708
Skew:                           8.030   Prob(JB):                         0.00
Kurtosis:   

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df[list(df_.columns[:-2])]

vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

,VIF,Column
9,258.851284,skill_ball_control
3,226.854632,attacking_short_passing
10,176.960366,movement_acceleration
5,164.279409,skill_dribbling
11,149.152887,movement_sprint_speed
13,101.279694,movement_reactions
12,96.509264,movement_agility
8,95.356802,skill_long_passing
25,88.517543,mentality_composure
15,77.531375,power_shot_power


In [ ]:
df_.describe()

,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,...,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle
count,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,...,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000
mean,49.718405,45.590218,52.221468,58.748003,42.809388,55.596509,47.325692,42.708885,52.768848,58.461976,...,46.812945,55.742149,46.380239,50.072163,53.609749,48.383357,58.528778,46.848889,47.640333,45.606631
std,18.325403,19.594609,17.428429,14.679653,17.701815,18.929118,18.422991,17.433612,15.246454,16.741878,...,19.322343,17.318157,20.775812,19.594022,13.955626,15.708099,11.880840,20.091287,21.585641,21.217734
min,5.000000,2.000000,5.000000,7.000000,3.000000,4.000000,6.000000,4.000000,8.000000,5.000000,...,4.000000,9.000000,3.000000,2.000000,9.000000,7.000000,12.000000,1.000000,5.000000,3.000000
25%,38.000000,30.000000,44.000000,54.000000,30.000000,50.000000,34.000000,31.000000,43.000000,54.000000,...,32.000000,44.000000,25.000000,39.000000,44.000000,39.000000,51.000000,29.000000,27.000000,24.000000
50%,54.000000,49.000000,56.000000,62.000000,44.000000,61.000000,49.000000,41.000000,56.000000,63.000000,...,51.000000,58.000000,52.000000,55.000000,55.000000,49.000000,60.000000,52.000000,55.000000,52.000000
75%,64.000000,62.000000,64.000000,68.000000,56.000000,68.000000,62.000000,56.000000,64.000000,69.000000,...,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,67.000000,64.000000,66.000000,64.000000
max,93.000000,95.000000,93.000000,92.000000,90.000000,97.000000,94.000000,94.000000,92.000000,96.000000,...,94.000000,95.000000,92.000000,95.000000,94.000000,92.000000,96.000000,94.000000,92.000000,90.000000


In [123]:
formula4 = ('wage_eur ~ defending_marking + skill_fk_accuracy + attacking_volleys + power_jumping + power_strength + mentality_aggression +'
               'attacking_heading_accuracy + mentality_composure + power_shot_power +power_long_shots ')
results4 = smf.ols(formula4, data=df).fit()
results4.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               wage_eur   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.199
Method:                 Least Squares   F-statistic:                     455.8
Date:                Thu, 24 Nov 2022   Prob (F-statistic):               0.00
Time:                        13:18:36   Log-Likelihood:            -2.0611e+05
No. Observations:               18278   AIC:                         4.122e+05
Df Residuals:                   18267   BIC:                         4.123e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -4.833e+04   1147.108    -42.134      0.000   -5.06e+04   -4.61e+04
defending_marking             73.9095     11.905      6.208      0.000      50.574      97.245
skill_fk_accuracy            -26.7993     15.088     -1.776      0.076     -56.374       2.775
attacking_volleys             99.0583     18.555      5.339      0.000      62.689     135.427
power_jumping                 73.3139     13.338      5.497      0.000      47.170      99.457
power_strength                30.9929     14.932      2.076      0.038       1.726      60.260
mentality_aggression         -55.1528     14.932     -3.694      0.000     -84.422     -25.884
attacking_heading_accuracy   -76.4616     14.755     -5.182      0.000    -105.383     -47.540
mentality_composure          701.7106     18.493     37.945      0.000     665.463     737.958
power_shot_power             365.9300     19.797     18.484      0.000     327.126     404.734
power_long_shots            -231.6539     19.383    -11.951      0.000    -269.647    -193.661
==============================================================================
Omnibus:                    25277.067   Durbin-Watson:                   0.687
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10102206.488
Skew:                           7.939   Prob(JB):                         0.00
Kurtosis:                     117.073   Cond. No.                     1.41e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.41e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [101]:
df_vif3 = calc_vif(df3)
df_vif3.sort_values(by= 'VIF', ascending=False)

,variables,VIF
6,mentality_aggression,37.057444
1,attacking_heading_accuracy,29.850998
5,power_strength,29.305086
0,attacking_crossing,26.933805
4,power_jumping,26.022986
2,attacking_volleys,25.836807
3,skill_fk_accuracy,22.515484
7,defending_marking,19.282565


In [103]:
#Y data is simple, do that first
y = np.array(df["wage_eur"]).reshape(-1,1)
y.shape

(18278, 1)

In [104]:
#Get a new df with only the features we'll use
df_ = df.drop(columns={"wage_eur"})
df_.head()

,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,...,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle
0,88,95,70,92,88,97,93,94,92,96,...,94,48,40,94,94,75,96,33,37,26
1,84,94,89,83,87,89,81,76,77,92,...,93,63,29,95,82,85,95,28,32,24
2,87,87,62,87,87,96,88,87,81,95,...,84,51,36,87,90,90,94,27,26,29
3,13,11,15,43,13,12,13,14,40,30,...,12,34,19,11,65,11,68,27,12,18
4,81,84,61,89,83,95,83,79,83,94,...,80,54,41,87,89,88,91,34,27,22


In [105]:
#Make that df into an array. 
x = np.array(df_)
x.shape

(18278, 29)

In [106]:
#shapes
print("X shape", x.shape)
print("Y shape", y.shape)

X shape (18278, 29)
Y shape (18278, 1)


In [107]:
#Split data
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size=.3)

In [108]:
print(xTrain.shape, yTrain.shape)
print(xTest.shape, yTest.shape)

(12794, 29) (12794, 1)
(5484, 29) (5484, 1)


In [109]:
#Generate model 
model = LinearRegression().fit(xTrain,yTrain)

In [110]:
#Get some info on our new regression model
r_sq = model.score(xTest, yTest)
print('R-squared:', r_sq)

R-squared: 0.27899435909278614


In [129]:
df4 = df[['defending_marking','skill_fk_accuracy','attacking_volleys','power_jumping','power_strength','mentality_aggression',
        'attacking_heading_accuracy','mentality_composure','power_shot_power','power_long_shots']]

In [130]:
#Make that df into an array. 
x1 = np.array(df4)
x1.shape

(18278, 10)

In [131]:
y1 = y
y1.shape

(18278, 1)

In [132]:
#shapes
print("X1 shape", x1.shape)
print("Y1 shape", y1.shape)

X1 shape (18278, 10)
Y1 shape (18278, 1)


In [133]:
x1Train, x1Test, y1Train, y1Test = train_test_split(x1,y1,test_size=.3)

In [134]:
print(x1Train.shape, y1Train.shape)
print(x1Test.shape, y1Test.shape)

(12794, 10) (12794, 1)
(5484, 10) (5484, 1)


In [135]:
#Generate model 
model = LinearRegression().fit(x1Train,y1Train)

In [136]:
#Get some info on our new regression model
r_sq1 = model.score(x1Test, y1Test)
print('R-squared:', r_sq1)

R-squared: 0.1893619995117619
